In [1]:
#new jupyter notebook

from tqdm import tqdm
from dataset import get_data_loader, move_data_to_device
from hparams import Hparams

train_loader = get_data_loader(split='train', args=Hparams.args)
for data in tqdm(train_loader):
    mel_spectrogram, yin, pyin = data#move_data_to_device(data, 'cpu')
    print(mel_spectrogram.shape)
    print(yin.shape)
    print(pyin.shape)
    #assert list(x.shape) == [8, 250, 256]  # shape in [B, T, D],
                                # i.e., [Batch size, num of frame per sample, spectrogram feature dimension]
    #assert list(onset.shape) == list(offset.shape) == list(octave.shape) == list(pitch_class.shape) == [8, 250]
    break
print('Congrats!')

  0%|          | 0/10 [00:03<?, ?it/s]

torch.Size([8, 256, 5])
torch.Size([8, 256])
torch.Size([8, 256])
Congrats!
